## Download data from the web

https://mm.kaist.ac.kr/datasets/voxceleb/index.html#testlist

download test data

```bash
curl -O https://mm.kaist.ac.kr/datasets/voxceleb/data/vox2_test_txt.zip
mkdir -p ../data/test
unzip vox2_test_txt.zip -d ../data/test
```

download dev data

```bash
curl -O https://mm.kaist.ac.kr/datasets/voxceleb/data/vox2_dev_txt.zip
mkdir -p ../data/dev
unzip vox2_dev_txt.zip -d ../data/dev
```

In [1]:
from pathlib import Path
import pandas as pd
import time


def get_df(dir: str) -> pd.DataFrame:
    """Create a DataFrame from downloaded vox celeb txt files.
    Dataframe contains following columns:
    speaker: speaker id,
    video: video id,
    txt_file: txt file name,
    start_time: start frame of the video,
    end_time: end frame of the video,
    length: length of the video in seconds.

    Args:
        dir (str): path to txt folder
    """
    data = []
    for speaker_dir in Path(dir).iterdir():
        if not speaker_dir.is_dir():
            continue
        for video_dir in speaker_dir.iterdir():
            if not video_dir.is_dir():
                continue
            for txt_file in video_dir.iterdir():

                with open(txt_file, "r") as f:
                    text = f.read()
                # read 8. line
                start_text = text.split("\n")[7]
                # get start time
                start_time = start_text.split(" ")[0]
                start_time = int(start_time)
                # get end time from last line
                end_time = text.split("\n")[-2].split(" ")[0]
                end_time = int(end_time)
                data.append(
                    (
                        speaker_dir.name,
                        video_dir.name,
                        txt_file.name,
                        start_time,
                        end_time,
                    )
                )

    df = pd.DataFrame(
        data, columns=["speaker", "video", "txt_file", "start_time", "end_time"]
    )
    df["length"] = (df["end_time"] - df["start_time"]) / 25

    # sort by speaker and length
    return df.sort_values(by=["speaker", "length"], ascending=[True, False])

In [2]:
import yt_dlp
from pydub import AudioSegment


def download_audio(youtube_url, output_path="audio.mp3"):
    """Download audio from a YouTube video and save it to a file."""
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_path,
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "192",
            }
        ],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    print(f"Audio downloaded as {output_path}")


def check_video_available(youtube_url):
    """Check if a YouTube video is available."""
    ydl_opts = {
        "quiet": True,
        "no_warnings": True,
        "format": "bestaudio/best",
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=False)
            return True
    except yt_dlp.utils.DownloadError as e:
        return
    return info is not None


def trim_audio(input_path, output_path, start_time, end_time):
    """Trim an audio file between start_time and end_time (in seconds)."""

    # Load the audio file
    audio = AudioSegment.from_file(input_path)

    # Convert times to milliseconds
    trimmed_audio = audio[start_time * 1000 : end_time * 1000]

    # Export the trimmed audio
    trimmed_audio.export(output_path, format="mp3")
    print(f"Trimmed audio saved as {output_path}")

In [3]:
def download_df_audio(df: pd.DataFrame, out_dir: str):
    """Download audio files from youtube urls in the DataFrame.
    DataFrame should contain columns: speaker, video, txt_file, start_time, end_time.

    Args:
        df (pd.DataFrame): DataFrame containing the data.
        out_dir (str): Output directory for
    """

    active_speaker = None
    """Variable to keep track of the current speaker."""
    downloaded_samples = 0
    """Variable to keep track of the number of samples downloaded for the current speaker."""
    last_video = None
    """Variable to keep track of the last video downloaded for the current speaker."""

    Path(out_dir).mkdir(parents=True, exist_ok=True)

    # sort by speaker and length
    df = df.sort_values(by=["speaker", "length"], ascending=[True, False])

    for index, row in df.iterrows():
        speaker = row["speaker"]
        video = row["video"]
        txt_file = row["txt_file"].replace(".txt", "")
        start_time = row["start_time"]
        end_time = row["end_time"]

        if active_speaker != speaker:
            active_speaker = speaker
            downloaded_samples = 0

        # skip if the same video for the same speaker
        if last_video == video:
            continue

        # skip if already downloaded 2 samples
        if downloaded_samples >= 2:
            continue

        print(f"Speaker: {speaker}, Video: {video}, Txt file: {txt_file}")

        # get youtube url
        youtube_url = f"https://www.youtube.com/watch?v={video}"
        output_name = f"{speaker}_{downloaded_samples}_{video}_{txt_file}"
        output_path = f"{output_name}.mp3"

        if not check_video_available(youtube_url):
            continue

        # download audio
        for i in range(3):
            try:
                download_audio(youtube_url, output_name)
                break
            except Exception as e:
                print(f"Failed to download audio: {e}")
                # wait for 5 seconds
                time.sleep(5)
            if i == 2:
                raise Exception("Failed to download audio after 3 attempts")

        # cut the audio
        # !ffmpeg -i {output_path} -ss {start_time} -to {end_time} -c copy {speaker}_{video}_cut.mp3
        trim_audio(
            output_path,
            f"{out_dir}/{output_name}_cut.mp3",
            start_time / 25,
            end_time / 25,
        )

        Path(output_path).unlink()
        downloaded_samples += 1
        last_video = video

## Test dataset

In [ ]:
IN_FOLDER = "../data/test/txt"

df_test = get_df(IN_FOLDER)
df_test

In [ ]:
df_test.to_csv("../data/test.csv", index=False)

In [ ]:
download_df_audio(df_test, "../data/test_data")

# Dev dataset

In [ ]:
df_dev = get_df("../data/dev/txt/")
df_dev

In [ ]:
df_dev.to_csv("../data/dev.csv", index=False)

## Download Most Famous Celebrities

In [4]:
import pandas as pd

NUM_OF_SAMPLES = 2
NUM_OF_CELEBS = 200

In [5]:
df_meta = pd.read_csv("../data/vox2_meta.csv")

# get NUM_OF_CELEBS celebs from dev set
df_meta = df_meta[df_meta["Set"] == "dev"]
df_meta = df_meta.sort_values(by="wiki_views_2024", ascending=False).head(NUM_OF_CELEBS)
df_meta

,Name,VoxCeleb2_ID,VGGFace2_ID,Gender,Set,wiki_views_2024
3031,Kamala_Harris,id04538,n004537,f,dev,30250192.0
4051,Mike_Tyson,id06121,n006120,m,dev,11900865.0
761,Bob_Marley,id01109,n001109,m,dev,10636588.0
3974,Michael_Jackson,id06004,n006003,m,dev,9373493.0
504,Ariana_Grande,id00752,n000752,f,dev,9362854.0
...,...,...,...,...,...,...
4052,Mikel_Arteta,id06122,n006121,m,dev,2030876.0
4980,Ronnie_Coleman,id07543,n007542,m,dev,2025382.0
4540,Peggy_Lipton,id06908,n006907,f,dev,2021658.0
2406,Jackie_Robinson,id03699,n003698,m,dev,2019565.0


In [6]:
df_dev = pd.read_csv("../data/dev.csv")

# limit df_dev to celebs in df_meta
df_top_celebs = df_dev[df_dev["speaker"].isin(df_meta["VoxCeleb2_ID"])]
df_top_celebs["speaker"].nunique()

200

In [7]:
df_top_celebs

,speaker,video,txt_file,start_time,end_time,length
1235,id00020,iRsViE6yAeg,00307.txt,1592,2053,18.44
1236,id00020,iRsViE6yAeg,00311.txt,3284,3741,18.28
1237,id00020,IpLYBI5Kaow,00064.txt,4414,4821,16.28
1238,id00020,l9PiLBUrSGs,00354.txt,8348,8750,16.08
1239,id00020,MM1fE2IkaDQ,00096.txt,2292,2648,14.24
...,...,...,...,...,...,...
1082801,id09185,AoO24VZf4oM,00069.txt,2827,2927,4.00
1082802,id09185,NDKxz17BhdI,00082.txt,6795,6895,4.00
1082803,id09185,nHM0sPZFkXQ,00154.txt,618,717,3.96
1082804,id09185,5TZfJJFAOwo,00031.txt,11828,11927,3.96


In [8]:
download_df_audio(df_top_celebs, "../data/download_df_audio")

Speaker: id00020, Video: iRsViE6yAeg, Txt file: 00307
[youtube] Extracting URL: https://www.youtube.com/watch?v=iRsViE6yAeg
[youtube] iRsViE6yAeg: Downloading webpage
[youtube] iRsViE6yAeg: Downloading tv client config
[youtube] iRsViE6yAeg: Downloading player 5ae7d525
[youtube] iRsViE6yAeg: Downloading tv player API JSON
[youtube] iRsViE6yAeg: Downloading ios player API JSON
[youtube] iRsViE6yAeg: Downloading m3u8 information
[info] iRsViE6yAeg: Downloading 1 format(s): 140
[download] Destination: id00020_0_iRsViE6yAeg_00307
[download] 100% of    3.43MiB in 00:00:00 at 5.09MiB/s   
[FixupM4a] Correcting container of "id00020_0_iRsViE6yAeg_00307"
[ExtractAudio] Destination: id00020_0_iRsViE6yAeg_00307.mp3
Deleting original file id00020_0_iRsViE6yAeg_00307 (pass -k to keep)
Audio downloaded as id00020_0_iRsViE6yAeg_00307
Trimmed audio saved as ../data/download_df_audio/id00020_0_iRsViE6yAeg_00307_cut.mp3
Speaker: id00020, Video: IpLYBI5Kaow, Txt file: 00064


ERROR: [youtube] IpLYBI5Kaow: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Speaker: id00020, Video: l9PiLBUrSGs, Txt file: 00354
[youtube] Extracting URL: https://www.youtube.com/watch?v=l9PiLBUrSGs
[youtube] l9PiLBUrSGs: Downloading webpage
[youtube] l9PiLBUrSGs: Downloading tv client config
[youtube] l9PiLBUrSGs: Downloading player 5ae7d525
[youtube] l9PiLBUrSGs: Downloading tv player API JSON
[youtube] l9PiLBUrSGs: Downloading ios player API JSON
[youtube] l9PiLBUrSGs: Downloading m3u8 information
[info] l9PiLBUrSGs: Downloading 1 format(s): 251
[download] Destination: id00020_1_l9PiLBUrSGs_00354
[download] 100% of    6.85MiB in 00:00:00 at 8.54MiB/s   
[ExtractAudio] Destination: id00020_1_l9PiLBUrSGs_00354.mp3
Deleting original file id00020_1_l9PiLBUrSGs_00354 (pass -k to keep)
Audio downloaded as id00020_1_l9PiLBUrSGs_00354
Trimmed audio saved as ../data/download_df_audio/id00020_1_l9PiLBUrSGs_00354_cut.mp3
Speaker: id00068, Video: 87KfuH7sucE, Txt file: 00001
[youtube] Extracting URL: https://www.youtube.com/watch?v=87KfuH7sucE
[youtube] 87KfuH7sucE: D

ERROR: [youtube] Sj8Fwx1C-_w: Video unavailable


Speaker: id00282, Video: Sj8Fwx1C-_w, Txt file: 00159


...

Speaker: id09171, Video: MWsO3D8uCMo, Txt file: 00174
[youtube] Extracting URL: https://www.youtube.com/watch?v=MWsO3D8uCMo
[youtube] MWsO3D8uCMo: Downloading webpage
[youtube] MWsO3D8uCMo: Downloading tv client config
[youtube] MWsO3D8uCMo: Downloading player 5ae7d525
[youtube] MWsO3D8uCMo: Downloading tv player API JSON
[youtube] MWsO3D8uCMo: Downloading ios player API JSON
[youtube] MWsO3D8uCMo: Downloading m3u8 information
[info] MWsO3D8uCMo: Downloading 1 format(s): 140
[download] Destination: id09171_1_MWsO3D8uCMo_00174
[download] 100% of    4.92MiB in 00:00:01 at 4.06MiB/s   
[FixupM4a] Correcting container of "id09171_1_MWsO3D8uCMo_00174"
[ExtractAudio] Destination: id09171_1_MWsO3D8uCMo_00174.mp3
Deleting original file id09171_1_MWsO3D8uCMo_00174 (pass -k to keep)
Audio downloaded as id09171_1_MWsO3D8uCMo_00174
Trimmed audio saved as ../data/download_df_audio/id09171_1_MWsO3D8uCMo_00174_cut.mp3
Speaker: id09179, Video: qeoRyOh5EUk, Txt file: 00067


ERROR: [youtube] qeoRyOh5EUk: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Speaker: id09179, Video: TY8kMus6dpw, Txt file: 00032
[youtube] Extracting URL: https://www.youtube.com/watch?v=TY8kMus6dpw
[youtube] TY8kMus6dpw: Downloading webpage
[youtube] TY8kMus6dpw: Downloading tv client config
[youtube] TY8kMus6dpw: Downloading player 5ae7d525
[youtube] TY8kMus6dpw: Downloading tv player API JSON
[youtube] TY8kMus6dpw: Downloading ios player API JSON
[youtube] TY8kMus6dpw: Downloading m3u8 information
[info] TY8kMus6dpw: Downloading 1 format(s): 251
[download] Destination: id09179_0_TY8kMus6dpw_00032
[download] 100% of    1.32MiB in 00:00:00 at 3.05MiB/s   
[ExtractAudio] Destination: id09179_0_TY8kMus6dpw_00032.mp3
Deleting original file id09179_0_TY8kMus6dpw_00032 (pass -k to keep)
Audio downloaded as id09179_0_TY8kMus6dpw_00032
Trimmed audio saved as ../data/download_df_audio/id09179_0_TY8kMus6dpw_00032_cut.mp3
Speaker: id09179, Video: PisbJ7IjIWA, Txt file: 00021


ERROR: [youtube] PisbJ7IjIWA: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Speaker: id09179, Video: PxTbEzlO-pU, Txt file: 00022
[youtube] Extracting URL: https://www.youtube.com/watch?v=PxTbEzlO-pU
[youtube] PxTbEzlO-pU: Downloading webpage
[youtube] PxTbEzlO-pU: Downloading tv client config
[youtube] PxTbEzlO-pU: Downloading player 5ae7d525
[youtube] PxTbEzlO-pU: Downloading tv player API JSON
[youtube] PxTbEzlO-pU: Downloading ios player API JSON
[youtube] PxTbEzlO-pU: Downloading m3u8 information
[info] PxTbEzlO-pU: Downloading 1 format(s): 140
[download] Destination: id09179_1_PxTbEzlO-pU_00022
[download] 100% of    7.02MiB in 00:00:00 at 8.10MiB/s   
[FixupM4a] Correcting container of "id09179_1_PxTbEzlO-pU_00022"
[ExtractAudio] Destination: id09179_1_PxTbEzlO-pU_00022.mp3
Deleting original file id09179_1_PxTbEzlO-pU_00022 (pass -k to keep)
Audio downloaded as id09179_1_PxTbEzlO-pU_00022
Trimmed audio saved as ../data/download_df_audio/id09179_1_PxTbEzlO-pU_00022_cut.mp3
Speaker: id09185, Video: hbnt54GExKU, Txt file: 00145
[youtube] Extracting URL: htt

## Download all Dev audio files

In [ ]:
import pandas as pd

df = pd.read_csv("../data/dev.csv")
df = df.sort_values(by=["speaker", "length"], ascending=[True, False])
# split df by speaker id to batches of 100 ids

# Get unique speakers
unique_speakers = df["speaker"].unique()

# Define batch size
batch_size = 100

# Create batches
batches = [
    unique_speakers[i : i + batch_size]
    for i in range(0, len(unique_speakers), batch_size)
]

# Create a DataFrame for each batch
batch_dfs = [df[df["speaker"].isin(batch)] for batch in batches]

In [ ]:
# TODO set start_batch
start_batch = 2
for i, batch_df in enumerate(batch_dfs):
    if i < start_batch:
        continue

    out_dir = f"../data/dev_data/batch_{i}"
    download_df_audio(batch_df, out_dir)